# Traces for the previous choice datan
Here we use the components extracted in the previous step and the timestamped data from the behavior

In [1]:
# Load the libraries
import caiman as cm
from caiman.source_extraction import cnmf

import matplotlib.pyplot as plt
from matplotlib.widgets import Button, TextBox
from matplotlib.patches import Polygon
from matplotlib.path import Path
import matplotlib.gridspec as gridspec
from matplotlib.widgets import CheckButtons

import numpy as np
import pandas as pd
import json
from scipy.sparse import csc_matrix
from scipy.stats import sem

from IPython import get_ipython
import IPython

import math

import os
from ipyfilechooser import FileChooser

from scipy.ndimage import binary_erosion

try:
    if __IPYTHON__:
        get_ipython().run_line_magic('load_ext', 'autoreload')
        get_ipython().run_line_magic('autoreload', '2')
except NameError:
    pass

## Load the data & create a mega data structure 

In [19]:
# Load the data
path = 'D:\\OneDrive - Rutgers University\\CaIm\\discr\\';
mouse_ids = ['os66', 'os70', 'os73']
data = {}
data_template = {'behav': [], 'licks_L': [], 'licks_R': [],
                 'traces': [], 'neu_qual': [], 'frame_timings': []}
for id in mouse_ids:
    path_mouse = path + id + '\\'
    # get all the dates for different experiements
    entries = os.listdir(path_mouse)
    dates = [entry for entry in entries if os.path.isdir(os.path.join(path_mouse, entry))]
    dates_dict = {}
    for date in dates:
        experiment = {}
        # get the file with behavioral data
        path_behav = path_mouse + date + '\\' + id + '_discr_' + date + '_behav.csv'
        behav = pd.read_csv(path_behav)
        licks_L = pd.read_csv(path_behav[0:-10] + '_licks_L.csv', header = None)
        licks_R = pd.read_csv(path_behav[0:-10] + '_licks_R.csv', header = None)
        experiment['behav'] = behav
        experiment['licks_L'] = licks_L
        experiment['licks_R'] = licks_R
        # get the neuronal traces
        path_neu = path_mouse + date + '\\' + 'good_neu_traces.csv'
        neu_traces = pd.read_csv(path_neu)
        neu_traces = neu_traces.to_numpy()
        experiment['traces'] = neu_traces
        # add all the experiment data 
        dates_dict[date] = experiment
    data[id] = dates_dict

## Possibly correct the timing

In [ ]:
# code for correcting the timing in the behavioral data from the videos

In [ ]:
## 

In [ ]:


# Example DataFrames
# neuron_traces = pd.DataFrame(...)  # Your neuron traces data
# data = pd.DataFrame(...)  # Your laser event data

# Sample neuron to plot
#neuron_id = 2  # Adjust this based on the neuron you want to plot

# Time conversion
sampling_rate = 10  # Hz
pre_event_points = 500 // (1000 / sampling_rate)  # 500 ms before
post_event_points = 3000 // (1000 / sampling_rate)  # 3000 ms after

# Function to extract data around laser event
def extract_data(side, n_id):
    times = data[data['side'] == side]['tone_ts']
    indices = (times * sampling_rate / 1000).astype(int)
    traces = []
    for idx in indices:
        start = int(idx - pre_event_points)
        end = int( idx + post_event_points)
        if start > 500 & end < neuron_traces.shape[1]-1:
            trace = neuron_traces[n_id, start:end]
            if trace.shape[0] == pre_event_points + post_event_points:
                traces.append(trace)
    return np.array(traces)


def plot_neu(neuron_id, ax_n):
    for i in [0,1,2]:
        # Extract data for each laser type
        traces_1 = extract_data(1, neuron_id+i)

        traces_0 = extract_data(-1, neuron_id+i)
        # Mean and std
        mean_trace_1 = np.mean(traces_1, axis=0)
        std_trace_1 = sem(traces_1, axis=0)

        mean_trace_0 = np.mean(traces_0, axis=0)
        std_trace_0 = sem(traces_0, axis=0)
        # Laser = 1
        axs[ax_n,i].plot(time_points, mean_trace_1, color='r', label='Mean')
        axs[ax_n,i].fill_between(time_points, mean_trace_1 - std_trace_1, mean_trace_1 + std_trace_1, color='r', alpha=0.2)

        # Laser = 0
        axs[ax_n,i].plot(time_points, mean_trace_0, color='k', label='Mean')
        axs[ax_n,i].fill_between(time_points, mean_trace_0 - std_trace_0, mean_trace_0 + std_trace_0, color='k', alpha=0.2)
        axs[ax_n,i].set_title('Neuron '+ str(neuron_id+i))



# Time points for x-axis
time_points = np.arange(-pre_event_points, post_event_points) * (1000 / sampling_rate)

# Plotting
fig, axs = plt.subplots(10, 3, figsize=(10, 15))
neu_indx = np.arange(0,30,3)
for n in np.arange(len(neu_indx)):
    plot_neu(neu_indx[n], n)


plt.show()






In [ ]:

%matplotlib widget
# Example DataFrames
# neuron_traces = pd.DataFrame(...)  # Your neuron traces data
# data = pd.DataFrame(...)  # Your laser event data

# Sample neuron to plot
#neuron_id = 2  # Adjust this based on the neuron you want to plot

# Time conversion
sampling_rate = 10  # Hz
pre_event_points = 1000 // (1000 / sampling_rate)  # 500 ms before
post_event_points = 4000 // (1000 / sampling_rate)  # 3000 ms after

# Function to extract data around laser event
def extract_data(side, n_id):
    times = data[data['side'] == side]['tone_ts']
    indices = (times * sampling_rate / 1000).astype(int)
    traces = []
    for idx in indices:
        start = int(idx - pre_event_points)
        end = int( idx + post_event_points)
        if start > 500 & end < neuron_traces.shape[1]-1:
            trace = neuron_traces[n_id, start:end]
            if trace.shape[0] == pre_event_points + post_event_points:
                traces.append(trace)
    return np.array(traces)


def plot_neu(neuron_id, ax_n, cols):
    for i in np.arange(cols):
        if neuron_id+i>=neuron_traces.shape[0]:
            break
        # Extract data for each laser type
        traces_1 = extract_data(1, neuron_id+i)

        traces_0 = extract_data(-1, neuron_id+i)
        # Mean and std
        mean_trace_1 = np.mean(traces_1, axis=0)
        std_trace_1 = sem(traces_1, axis=0)

        mean_trace_0 = np.mean(traces_0, axis=0)
        std_trace_0 = sem(traces_0, axis=0)
        # Laser = 1
        axs[ax_n,i].plot(time_points, mean_trace_1, color='r', label='Mean')
        axs[ax_n,i].fill_between(time_points, mean_trace_1 - std_trace_1, mean_trace_1 + std_trace_1, color='r', alpha=0.2)

        # Laser = 0
        axs[ax_n,i].plot(time_points, mean_trace_0, color='b', label='Mean')
        axs[ax_n,i].fill_between(time_points, mean_trace_0 - std_trace_0, mean_trace_0 + std_trace_0, color='b', alpha=0.2)
        axs[ax_n,i].set_title('Neuron '+ str(neuron_id+i))

        # Lines at start tone, start response
        min_v = np.amin([np.amin(mean_trace_1), np.amin(mean_trace_0)])
        max_v = np.amax([np.amax(mean_trace_1), np.amax(mean_trace_0)])
        maxes = [np.amin(mean_trace_1), np.amax(mean_trace_1)]
        axs[ax_n,i].plot([0,0], [min_v, max_v], color='k', label='tone')
        axs[ax_n,i].plot([1150,1150], [min_v, max_v], color='k', label='RESPONSE')

        # Create a small axis for the checkbox next to each subplot
        bbox = axs[ax_n, i].get_position()

        ax_check = fig.add_axes([bbox.xmax, bbox.ymax-0.01, 0.04, 0.01])
        
        checkbox = CheckButtons(ax_check, [str(neuron_id+i)], [False])
        checkbox_axes.append(checkbox)



# Time points for x-axis
time_points = np.arange(-pre_event_points, post_event_points) * (1000 / sampling_rate)





# Plotting
cols = 4
nneu = neuron_traces.shape[0]
rows = math.ceil(nneu/cols)
fig, axs = plt.subplots(rows, cols, figsize=(10, 30))

checkbox_axes = []

neu_indx = np.arange(0,nneu,cols)
for n in np.arange(len(neu_indx)):
    plot_neu(neu_indx[n], n, cols)
    


plt.show()

relevant_neu = []

def check_handler(label):
    neuron_id = int(label)
    if neuron_id in relevant_neu:
        relevant_neu.remove(neuron_id)
    else:
        relevant_neu.append(neuron_id)

# Attach the handler to each checkbox
for check in checkbox_axes:
    check.on_clicked(check_handler)

In [ ]:
relevant_neu = []

# Attach the handler to each checkbox
n = 0
for check in checkbox_axes:
    if check.get_status()==[True]:
        relevant_neu.append(n)
    n+=1


In [ ]:
len(relevant_neu)

# For each side, select prev_same and prev_diff

In [ ]:
#data['prev_same'] = 
prev_same=[data['side'][n]==data['side'][n-1] for n in np.arange(1,data.shape[0])]

start=[False]
start.extend(prev_same)
prev_same = start
data['prev_same'] = prev_same
data.head()

# Plot prev same and prev diff for the left side and right sides

In [ ]:


# Example DataFrames
# neuron_traces = pd.DataFrame(...)  # Your neuron traces data
# data = pd.DataFrame(...)  # Your laser event data

# Sample neuron to plot
#neuron_id = 2  # Adjust this based on the neuron you want to plot

# Time conversion
sampling_rate = 10  # Hz
pre_event_points = 1000 // (1000 / sampling_rate)  # 500 ms before
post_event_points = 4000 // (1000 / sampling_rate)  # 3000 ms after

# Function to extract data around laser event
def extract_data(side, prev_same, n_id):
    times = data[(data['side'] == side) & (data['prev_same'] == prev_same)]['tone_ts']
    indices = (times * sampling_rate / 1000).astype(int)
    traces = []
    for idx in indices:
        start = int(idx - pre_event_points)
        end = int( idx + post_event_points)
        if start > 500 & end < neuron_traces.shape[1]-1:
            trace = neuron_traces[n_id, start:end]
            if trace.shape[0] == pre_event_points + post_event_points:
                traces.append(trace)
    return np.array(traces)


def plot_neu(neuron_id, ax_n):
    # Left side
    side = 1
    traces_L_same = extract_data(side, True, neuron_id)
    traces_L_diff = extract_data(side, False, neuron_id)
    # Mean and std
    mean_trace_L_same = np.mean(traces_L_same, axis=0)
    std_trace_L_same = sem(traces_L_same, axis=0)
    mean_trace_L_diff = np.mean(traces_L_diff, axis=0)
    std_trace_L_diff = sem(traces_L_diff, axis=0)

    # Plot 
    axs[ax_n,0].plot(time_points, mean_trace_L_same, color='b', label='Same')
    axs[ax_n,0].fill_between(time_points, mean_trace_L_same - std_trace_L_same, mean_trace_L_same + std_trace_L_same, color='b', alpha=0.2)
    axs[ax_n,0].plot(time_points, mean_trace_L_diff, color='r', label='Diff')
    axs[ax_n,0].fill_between(time_points, mean_trace_L_diff - std_trace_L_diff, mean_trace_L_diff + std_trace_L_diff, color='r', alpha=0.2)
    axs[ax_n,0].set_title('Neuron '+ str(neuron_id) + ', left')

        # Lines at start tone, start response
    min_v = np.amin([np.amin(mean_trace_L_same), np.amin(mean_trace_L_diff)])
    max_v = np.amax([np.amax(mean_trace_L_same), np.amax(mean_trace_L_diff)])
    axs[ax_n,0].plot([0,0], [min_v, max_v], color='k', label='tone')
    axs[ax_n,0].plot([1150,1150], [min_v, max_v], color='k', label='RESPONSE')
    
    # Right side
    side = -1
    traces_L_same = extract_data(side, True, neuron_id)
    traces_L_diff = extract_data(side, False, neuron_id)
    # Mean and std
    mean_trace_L_same = np.mean(traces_L_same, axis=0)
    std_trace_L_same = sem(traces_L_same, axis=0)
    mean_trace_L_diff = np.mean(traces_L_diff, axis=0)
    std_trace_L_diff = sem(traces_L_diff, axis=0)

    # Plot 
    axs[ax_n,1].plot(time_points, mean_trace_L_same, color='b', label='Same')
    axs[ax_n,1].fill_between(time_points, mean_trace_L_same - std_trace_L_same, mean_trace_L_same + std_trace_L_same, color='b', alpha=0.2)
    axs[ax_n,1].plot(time_points, mean_trace_L_diff, color='r', label='Diff')
    axs[ax_n,1].fill_between(time_points, mean_trace_L_diff - std_trace_L_diff, mean_trace_L_diff + std_trace_L_diff, color='r', alpha=0.2)
    axs[ax_n,1].set_title('Neuron '+ str(neuron_id) + ', right')
            # Lines at start tone, start response
    min_v = np.amin([np.amin(mean_trace_L_same), np.amin(mean_trace_L_diff)])
    max_v = np.amax([np.amax(mean_trace_L_same), np.amax(mean_trace_L_diff)])
    axs[ax_n,1].plot([0,0], [min_v, max_v], color='k', label='tone')
    axs[ax_n,1].plot([1150,1150], [min_v, max_v], color='k', label='RESPONSE')



# Time points for x-axis
time_points = np.arange(-pre_event_points, post_event_points) * (1000 / sampling_rate)

# Plotting

fig, axs = plt.subplots(len(relevant_neu), 2, figsize=(10, 70))

for n in np.arange(len(relevant_neu)):
    plot_neu(relevant_neu[n], n)

plt.show()

# Now, same but without errors

In [ ]:


# Example DataFrames
# neuron_traces = pd.DataFrame(...)  # Your neuron traces data
# data = pd.DataFrame(...)  # Your laser event data

# Sample neuron to plot
#neuron_id = 2  # Adjust this based on the neuron you want to plot

# Time conversion
sampling_rate = 10  # Hz
pre_event_points = 1000 // (1000 / sampling_rate)  # 500 ms before
post_event_points = 4000 // (1000 / sampling_rate)  # 3000 ms after

# Function to extract data around laser event
def extract_data(side, prev_same, n_id):
    times = data[(data['side'] == side) & (data['prev_same'] == prev_same) & (data['choice'] == side)]['tone_ts']
    indices = (times * sampling_rate / 1000).astype(int)
    traces = []
    for idx in indices:
        start = int(idx - pre_event_points)
        end = int( idx + post_event_points)
        if start > 500 & end < neuron_traces.shape[1]-1:
            trace = neuron_traces[n_id, start:end]
            if trace.shape[0] == pre_event_points + post_event_points:
                traces.append(trace)
    return np.array(traces)


def plot_neu(neuron_id, ax_n):
    # Left side
    side = 1
    traces_L_same = extract_data(side, True, neuron_id)
    traces_L_diff = extract_data(side, False, neuron_id)
    # Mean and std
    mean_trace_L_same = np.mean(traces_L_same, axis=0)
    std_trace_L_same = sem(traces_L_same, axis=0)
    mean_trace_L_diff = np.mean(traces_L_diff, axis=0)
    std_trace_L_diff = sem(traces_L_diff, axis=0)

    # Plot 
    axs[ax_n,0].plot(time_points, mean_trace_L_same, color='b', label='Same')
    axs[ax_n,0].fill_between(time_points, mean_trace_L_same - std_trace_L_same, mean_trace_L_same + std_trace_L_same, color='b', alpha=0.2)
    axs[ax_n,0].plot(time_points, mean_trace_L_diff, color='r', label='Diff')
    axs[ax_n,0].fill_between(time_points, mean_trace_L_diff - std_trace_L_diff, mean_trace_L_diff + std_trace_L_diff, color='r', alpha=0.2)
    axs[ax_n,0].set_title('Neuron '+ str(neuron_id) + ', left')

        # Lines at start tone, start response
    min_v = np.amin([np.amin(mean_trace_L_same), np.amin(mean_trace_L_diff)])
    max_v = np.amax([np.amax(mean_trace_L_same), np.amax(mean_trace_L_diff)])
    axs[ax_n,0].plot([0,0], [min_v, max_v], color='k', label='tone')
    axs[ax_n,0].plot([1150,1150], [min_v, max_v], color='k', label='RESPONSE')
    
    # Right side
    side = -1
    traces_L_same = extract_data(side, True, neuron_id)
    traces_L_diff = extract_data(side, False, neuron_id)
    # Mean and std
    mean_trace_L_same = np.mean(traces_L_same, axis=0)
    std_trace_L_same = sem(traces_L_same, axis=0)
    mean_trace_L_diff = np.mean(traces_L_diff, axis=0)
    std_trace_L_diff = sem(traces_L_diff, axis=0)

    # Plot 
    axs[ax_n,1].plot(time_points, mean_trace_L_same, color='b', label='Same')
    axs[ax_n,1].fill_between(time_points, mean_trace_L_same - std_trace_L_same, mean_trace_L_same + std_trace_L_same, color='b', alpha=0.2)
    axs[ax_n,1].plot(time_points, mean_trace_L_diff, color='r', label='Diff')
    axs[ax_n,1].fill_between(time_points, mean_trace_L_diff - std_trace_L_diff, mean_trace_L_diff + std_trace_L_diff, color='r', alpha=0.2)
    axs[ax_n,1].set_title('Neuron '+ str(neuron_id) + ', right')
            # Lines at start tone, start response
    min_v = np.amin([np.amin(mean_trace_L_same), np.amin(mean_trace_L_diff)])
    max_v = np.amax([np.amax(mean_trace_L_same), np.amax(mean_trace_L_diff)])
    axs[ax_n,1].plot([0,0], [min_v, max_v], color='k', label='tone')
    axs[ax_n,1].plot([1150,1150], [min_v, max_v], color='k', label='RESPONSE')



# Time points for x-axis
time_points = np.arange(-pre_event_points, post_event_points) * (1000 / sampling_rate)

# Plotting

fig, axs = plt.subplots(len(relevant_neu), 2, figsize=(10, 70))

for n in np.arange(len(relevant_neu)):
    plot_neu(relevant_neu[n], n)

plt.show()

# Now, no errors next to errors 

In [ ]:
# Function to extract data around laser event
def extract_data_side_prev_correct(side, prev_same, correct, n_id):
    if correct:
        choice = side
    else:
        choice = - side
    times = data[(data['side'] == side) & (data['prev_same'] == prev_same) & (data['choice'] == choice)]['tone_ts']
    indices = (times * sampling_rate / 1000).astype(int)
    traces = []
    for idx in indices:
        start = int(idx - pre_event_points)
        end = int( idx + post_event_points)
        if start > 500 & end < neuron_traces.shape[1]-1:
            trace = neuron_traces[n_id, start:end]
            if trace.shape[0] == pre_event_points + post_event_points:
                traces.append(trace)
    return np.array(traces)

In [ ]:
# Time conversion
sampling_rate = 10  # Hz
pre_event_points = 1000 // (1000 / sampling_rate)  # 500 ms before
post_event_points = 4000 // (1000 / sampling_rate)  # 3000 ms after

def plot_neu(neuron_id, ax_n):
    sides = [1,1,-1,-1]
    correctnesses = [True, False, True, False]
    titles = ['left correct','left incorrect','right correct','right incorrect']

    for i in [0,1,2,3]:
        side = sides[i]
        correctness = correctnesses[i]
        traces_same = extract_data_side_prev_correct(side, True, correctness, neuron_id)
        traces_diff = extract_data_side_prev_correct(side, False, correctness, neuron_id)
    
        # Mean and std
        mean_trace_same = np.mean(traces_same, axis=0)
        std_trace_same = sem(traces_same, axis=0)
        mean_trace_diff = np.mean(traces_diff, axis=0)
        std_trace_diff = sem(traces_diff, axis=0)

    # Plot 
        axs[ax_n,i].plot(time_points, mean_trace_same, color='b', label='Same')
        axs[ax_n,i].fill_between(time_points, mean_trace_same - std_trace_same, mean_trace_same + std_trace_same, color='b', alpha=0.2)
        axs[ax_n,i].plot(time_points, mean_trace_diff, color='r', label='Diff')
        axs[ax_n,i].fill_between(time_points, mean_trace_diff - std_trace_diff, mean_trace_diff + std_trace_diff, color='r', alpha=0.2)
        axs[ax_n,i].set_title('Neuron '+ str(neuron_id) + ', ' + titles[i])

        # Lines at start tone, start response
        min_v = np.amin([np.amin(mean_trace_same), np.amin(mean_trace_diff)])
        max_v = np.amax([np.amax(mean_trace_same), np.amax(mean_trace_diff)])
        axs[ax_n,i].plot([0,0], [min_v, max_v], color='k', label='tone')
        axs[ax_n,i].plot([1150,1150], [min_v, max_v], color='k', label='RESPONSE')


# Time points for x-axis
time_points = np.arange(-pre_event_points, post_event_points) * (1000 / sampling_rate)

# Plotting

fig, axs = plt.subplots(len(relevant_neu), 4, figsize=(10, 70))

for n in np.arange(len(relevant_neu)):
    plot_neu(relevant_neu[n], n)

plt.show()

In [ ]:
# Only errors

In [ ]:


# Example DataFrames
# neuron_traces = pd.DataFrame(...)  # Your neuron traces data
# data = pd.DataFrame(...)  # Your laser event data

# Sample neuron to plot
#neuron_id = 2  # Adjust this based on the neuron you want to plot

# Time conversion
sampling_rate = 10  # Hz
pre_event_points = 1000 // (1000 / sampling_rate)  # 500 ms before
post_event_points = 4000 // (1000 / sampling_rate)  # 3000 ms after

# Function to extract data around laser event
def extract_data(side, prev_same, n_id):
    times = data[(data['side'] == side) & (data['prev_same'] == prev_same) & (data['choice_made'] == -side)]['tone_ts']
    indices = (times * sampling_rate / 1000).astype(int)
    traces = []
    for idx in indices:
        start = int(idx - pre_event_points)
        end = int( idx + post_event_points)
        if start > 500 & end < neuron_traces.shape[1]-1:
            trace = neuron_traces[n_id, start:end]
            if trace.shape[0] == pre_event_points + post_event_points:
                traces.append(trace)
    return np.array(traces)


def plot_neu(neuron_id, ax_n):
    # Left side
    side = 1
    traces_L_same = extract_data(side, True, neuron_id)
    traces_L_diff = extract_data(side, False, neuron_id)
    # Mean and std
    mean_trace_L_same = np.mean(traces_L_same, axis=0)
    std_trace_L_same = sem(traces_L_same, axis=0)
    mean_trace_L_diff = np.mean(traces_L_diff, axis=0)
    std_trace_L_diff = sem(traces_L_diff, axis=0)

    # Plot 
    axs[ax_n,0].plot(time_points, mean_trace_L_same, color='b', label='Same')
    axs[ax_n,0].fill_between(time_points, mean_trace_L_same - std_trace_L_same, mean_trace_L_same + std_trace_L_same, color='b', alpha=0.2)
    axs[ax_n,0].plot(time_points, mean_trace_L_diff, color='r', label='Diff')
    axs[ax_n,0].fill_between(time_points, mean_trace_L_diff - std_trace_L_diff, mean_trace_L_diff + std_trace_L_diff, color='r', alpha=0.2)
    axs[ax_n,0].set_title('Neuron '+ str(neuron_id) + ', left')

        # Lines at start tone, start response
    min_v = np.amin([np.amin(mean_trace_L_same), np.amin(mean_trace_L_diff)])
    max_v = np.amax([np.amax(mean_trace_L_same), np.amax(mean_trace_L_diff)])
    axs[ax_n,0].plot([0,0], [min_v, max_v], color='k', label='tone')
    axs[ax_n,0].plot([1150,1150], [min_v, max_v], color='k', label='RESPONSE')
    
    # Right side
    side = -1
    traces_L_same = extract_data(side, True, neuron_id)
    traces_L_diff = extract_data(side, False, neuron_id)
    # Mean and std
    mean_trace_L_same = np.mean(traces_L_same, axis=0)
    std_trace_L_same = sem(traces_L_same, axis=0)
    mean_trace_L_diff = np.mean(traces_L_diff, axis=0)
    std_trace_L_diff = sem(traces_L_diff, axis=0)

    # Plot 
    axs[ax_n,1].plot(time_points, mean_trace_L_same, color='b', label='Same')
    axs[ax_n,1].fill_between(time_points, mean_trace_L_same - std_trace_L_same, mean_trace_L_same + std_trace_L_same, color='b', alpha=0.2)
    axs[ax_n,1].plot(time_points, mean_trace_L_diff, color='r', label='Diff')
    axs[ax_n,1].fill_between(time_points, mean_trace_L_diff - std_trace_L_diff, mean_trace_L_diff + std_trace_L_diff, color='r', alpha=0.2)
    axs[ax_n,1].set_title('Neuron '+ str(neuron_id) + ', right')
            # Lines at start tone, start response
    min_v = np.amin([np.amin(mean_trace_L_same), np.amin(mean_trace_L_diff)])
    max_v = np.amax([np.amax(mean_trace_L_same), np.amax(mean_trace_L_diff)])
    axs[ax_n,1].plot([0,0], [min_v, max_v], color='k', label='tone')
    axs[ax_n,1].plot([1150,1150], [min_v, max_v], color='k', label='RESPONSE')



# Time points for x-axis
time_points = np.arange(-pre_event_points, post_event_points) * (1000 / sampling_rate)

# Plotting

fig, axs = plt.subplots(len(relevant_neu), 2, figsize=(10, 70))

for n in np.arange(len(relevant_neu)):
    plot_neu(relevant_neu[n], n)

plt.show()

In [ ]:
data[(data['choice']==-1) & (data['side']==1)].shape

correct 
left: 78; right: 104
error
left: 13; right: 35



In [ ]:
relevant_neu
# Save the numbers of the good components
relevant_neu_df = pd.DataFrame(relevant_neu, columns=['relevant_neu'])
relevant_neu_df.to_csv(path + 'relevant_neu.csv', index=False)
relevant_neu_df

In [ ]:
path